In [4]:
import os
import time
import requests
from lumaai import LumaAI
from dotenv import load_dotenv
import json

In [ ]:
import os
import time
import requests
from lumaai import LumaAI
from dotenv import load_dotenv
import json
# Load environment variables
load_dotenv()

print(os.environ.get("LUMAAI_API_KEY"))

# Initialize the client with API key
client = LumaAI()


def test_image_generation():
    """Test image generation and return the image URL"""
    print("Starting image generation test...")

    # Create an image generation
    generation = client.generations.image.create(
        prompt=f"""Close-up portrait of a young woman with short blue hair, gazing out of a rain-streaked window, reflecting ambient neon city lights in the puddles of water. The artistic style is inspired by impressionist paintings.
""",
        aspect_ratio="16:9",
        model="photon-1"
    )

    # Poll for completion
    completed = False
    while not completed:
        generation = client.generations.get(id=generation.id)
        if generation.state == "completed":
            completed = True
        elif generation.state == "failed":
            raise RuntimeError(
                f"Generation failed: {generation.failure_reason}")
        print("Generating image...")
        time.sleep(2)

    print(f"Image generation completed! URL: {generation.assets.image}")
    return generation.assets.image


def test_video_generation(image_url):
    """Test video generation using the generated image"""
    print("\nStarting video generation test...")

    generation = client.generations.create(
        prompt="""Close-up portrait of a young woman with short blue hair, gazing out of a rain-streaked window, reflecting ambient neon city lights in the puddles of water. The artistic style is inspired by impressionist paintings.
        """,
        model="ray-flash-2",
        duration='5s',
        keyframes={
            "frame0": {
                "type": "image",
                "url": image_url
            }
        }
    )

    # Poll for completion
    completed = False
    while not completed:
        generation = client.generations.get(id=generation.id)
        if generation.state == "completed":
            completed = True
        elif generation.state == "failed":
            raise RuntimeError(
                f"Generation failed: {generation.failure_reason}")
        print("Generating video...")
        time.sleep(2)

    print(f"Video generation completed! URL: {generation.assets.video}")
    return generation, generation.assets.video


image_url = test_image_generation()

# Test video generation using the generated image
generation, video_url = test_video_generation(image_url)

print("\nAll tests completed successfully!")
print(f"Image URL: {image_url}")
print(f"Video URL: {video_url}")


print(generation)

luma-3e9ca934-dfa7-456f-a274-b850a3154b33-2f3350aa-8dc7-41f1-b05a-6261a28ea0b7
Starting image generation test...
Generating image...
Generating image...
Generating image...
Generating image...
Generating image...
Generating image...
Generating image...
Image generation completed! URL: https://storage.cdn-luma.com/dream_machine/af7d5238-3151-4b95-b6f1-cfcaa28a3916/59a8b8f5-46ff-45e6-bfa7-e21d5be3181f_result99f24cc9004001dd.jpg

Starting video generation test...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating video...
Generating vide

In [10]:
dict(generation)
dict(generation.assets)

{'image': 'https://storage.cdn-luma.com/dream_machine/25c75ec7-68c2-42cb-9916-786b1c5176f8/d369b4cc-c3da-4581-a374-7224e55a3a28_result0fb7ee1cb95f2b0a_thumb.jpg',
 'progress_video': None,
 'video': 'https://storage.cdn-luma.com/dream_machine/25c75ec7-68c2-42cb-9916-786b1c5176f8/10c4314a-2330-4bf3-88e9-47e13329b96e_result0fb7ee1cb95f2b0a.mp4'}

In [7]:


system_instructions = """You are a skilled music video concept generator. You will first analyze a song provided by the user. This analysis should include identifying the song's genre, tempo (approximate BPM), overall mood, key lyrical themes, and prominent instrumentation.

Based on your analysis, you will create three distinct scenes for a music video. For each scene, provide a detailed video prompt (suitable for a text-to-video AI model) and an image prompt (suitable for a text-to-image AI model). The image prompts must be crafted to visually support the corresponding video prompt.

Focus on creating scenes that enhance the atmosphere, character actions, and visual elements and is suitable for AI to make an image or video from.

The character description, if applicable, should be only three sentences long.

Your response MUST be provided as a valid JSON object with the following structure:
{
  "song_analysis": {
    "title": "Song title (guess if unknown)",
    "artist": "Artist name (guess if unknown)",
    "genre": "Primary genre of the song",
    "tempo_bpm": "Estimated tempo in BPM",
    "mood": "Overall mood/emotion of the song",
    "lyrical_themes": "Main themes in the lyrics (if present)",
    "instrumentation": "Main instruments heard",
    "character_description": "Brief character description for the main subject (2-3 sentences max)"
  },
  "scenes": [
    {
      "scene_number": 1,
      "scene_setting": "Location/environment",
      "video_prompt": "Detailed video prompt for scene",
      "image_prompt": "Detailed image prompt for a still from this scene",
      "scene_description": "Description of what happens",
      "action": "What the character(s) are doing"
    },
    {
      "scene_number": 2,
      "scene_setting": "...",
      "video_prompt": "...",
      "image_prompt": "...",
      "scene_description": "...",
      "action": "..."
    }
    // Generate 4-6 scenes total
  ]
}"""

prompt = f"""Analyze the following song and create a music video concept."""

In [10]:
from google import genai
import typing_extensions as typing
from google.genai import types
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any  # Add Optional, Dict, and Any
import re
import json
from pydantic import ValidationError


class SongAnalysis(BaseModel):
    title: str
    artist: str
    genre: str
    tempo_bpm: str = Field(..., description="Approximate BPM")
    mood: str
    lyrical_themes: str
    instrumentation: str
    character_description: str = Field(...,
                                       description="Brief character description (max 3 sentences)")


class Scene(BaseModel):
    scene_number: int
    scene_setting: str
    video_prompt: str
    image_prompt: str
    scene_description: str
    action: str


class MusicVideoScenes(BaseModel):
    song_analysis: SongAnalysis
    scenes: List[Scene]


load_dotenv()

client = genai.Client(api_key=os.environ.get("GEMINI_KEY"))

myfile = client.files.upload(file='Lofi Girl.mp3')

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        system_instruction=system_instructions),
    contents=[prompt, myfile]
)

json_str = response.text

try:
    # Check if the response contains markdown code blocks
    json_match = re.search(r'```json\s*(.*?)\s*```', response.text, re.DOTALL)

    if json_match:
        # Extract just the JSON without the markdown code block tags
        json_str = json_match.group(1)
    else:
        # If no markdown code blocks, use the full response
        json_str = response.text

    # Parse the JSON string
    music_video_data = json.loads(json_str)

    # Create a MusicVideoScenes instance
    music_video_scenes = MusicVideoScenes(**music_video_data)

    print("Successfully parsed response into MusicVideoScenes object!")
    print(f"Song title: {music_video_scenes.song_analysis.title}")
    print(f"Number of scenes: {len(music_video_scenes.scenes)}")

    # You can also save the entire structured data
    with open('music_video_storyboard.json', 'w') as f:
        json.dump(music_video_data, f, indent=2)

except json.JSONDecodeError as e:
    print("Error parsing JSON response:", e)
    print("Raw response:", response.text)
except ValidationError as e:
    print("Pydantic validation error:", e)
    print("JSON data received:", music_video_data)
except Exception as e:
    print("Error creating MusicVideoScenes object:", e)
    print("Exception:", str(e))

Successfully parsed response into MusicVideoScenes object!
Song title: Lo-fi Girl
Number of scenes: 3


In [ ]:
type

In [11]:
type(client)

google.genai.client.Client